## Tabular Playground Series March 2021

<img src="https://i.imgur.com/uHVJtv0.png">



<br><br>

### Notebook Contents:

0. [**Imports, Data Loading and Preprocessing**](#loading)

1. [**Optuna Hyperparameter Optimization**](#optuna)

2. [**Submission**](#submission)

In [3]:
pip install optuna

     |████████████████████████████████| 293 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 24.2 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 75 kB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 14.4 MB/s eta 0:00:01
  Using cached pbr-5.5.1-py2.py3-none-any.whl (106 kB)
     |████████████████████████████████| 133 kB 19.4 MB/s eta 0:00:01
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11107 sha256=195de666d711b65cd8d7466b7c46021178a63f2626c887c8b59cce1752cd418f
  Stored in directory: /Users/jsusser/Library/Caches/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
Note: you may need to restart the kernel to use updated packages.


<a id="loading"></a>

##### 0. Imports, Data Loading and Preprocessing

In [9]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 100
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest
import warnings
warnings.filterwarnings('ignore')
import optuna
import os
root_path = '../../input'

In [10]:
train = pd.read_csv(os.path.join(root_path, 'train.csv'))
test = pd.read_csv(os.path.join(root_path, 'test.csv'))
sample_submission = pd.read_csv(os.path.join(root_path, 'sample_submission.csv'))

categorical_cols = [i for i in train.columns if 'cat' in i]

dataset = pd.concat([train, test], axis = 0, ignore_index = True)
train_len = len(train)

In [11]:
float_cols = list(set(dataset.select_dtypes(['float']).columns.tolist()) - set(['target']))

In [12]:
for col in float_cols:
    transformer = QuantileTransformer(n_quantiles=100, 
                                      random_state=0, output_distribution="normal")   # from optimal commit 9
    data_len = len(dataset)
    raw_vec = dataset[col].values.reshape(data_len, 1)
    transformer.fit(raw_vec)
    dataset[col+"_qt"] = transformer.transform(raw_vec)

In [13]:
dataset = pd.get_dummies(dataset, columns = categorical_cols)
train_preprocessed = dataset.iloc[:train_len, ]
test_preprocessed = dataset.iloc[train_len:, ]

test_cols_always_0 = (test_preprocessed.drop('target',1).sum(axis = 0)
                      .rename("n_non_null").to_frame().query("n_non_null == 0").index.tolist())

features = list(set(train_preprocessed.drop(['id', 'target'], 1).columns.tolist()) - set(test_cols_always_0))

assert train_preprocessed.shape[1] == test_preprocessed.shape[1]

**Disclaimer:** 

I did not inspect whether some of the categorical columns in train have values not present in test or viceversa. A simple solution would be to drop them in train if you knew it beforehand.

<a id="optuna"></a>

### Optuna

Look [here](https://optuna.readthedocs.io/en/stable/tutorial/) for reference about Optuna library. 

Look [here](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html) for a set of Lightgbm Classifier hyperparameters.


Skip and go [here](#hyperparams) to find my best parameters.

In [14]:
#Set to False if you want to skip it

OPTUNA_OPTIMIZATION = True

In [15]:
N_SPLITS = 10 #Number of folds for validation
N_TRIALS = 50 #Number of trials to find best hyperparameters

In [16]:
def objective(trial, cv=StratifiedKFold(N_SPLITS, shuffle = True, random_state = 29)):
    
    
    param_lgb = {
        "random_state": trial.suggest_int("random_state", 1, 100),
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "max_depth": trial.suggest_int("max_depth", -1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    model = LGBMClassifier(**param_lgb)
    
    val_aucs = []
    aucs = []
    
    for kfold, (train_idx, val_idx) in enumerate(cv.split(train_preprocessed[features].values, train['target'].values)):
        
        model.fit(train_preprocessed.loc[train_idx, features], train_preprocessed.loc[train_idx, 'target'])
        print('Fitted {}'.format(type(model).__name__))
        val_true = train.loc[val_idx, 'target'].values
        
        preds = model.predict(train_preprocessed.loc[val_idx, features])
        
        auc = roc_auc_score(val_true, preds)
        
        print('Fold: {}\t AUC: {}\n'.format(kfold, auc))
        aucs.append(auc)
    
    print('Average AUC: {}'.format(np.average(auc)))
    return np.average(aucs)

In [ ]:
if OPTUNA_OPTIMIZATION:
    study = optuna.create_study(study_name = 'lgbm_parameter_opt', direction="maximize")
    study.optimize(objective, n_trials=N_TRIALS) 
    
    trial = study.best_trial
    
    print("  Value: {}".format(trial.value))
    
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
else:
    trial = {"reg_alpha": 0.362136938773081,
             "reg_lambda": 2.930297242488071,
             "max_depth": 10,
             "n_estimators": 306,
             "num_leaves": 71,
             "colsample_bytree": 0.7121396258381646,
             "subsample": 0.793959734582999,
             "subsample_freq": 2,
             "min_child_samples": 18}

[I 2021-03-22 14:02:51,857] A new study created in memory with name: lgbm_parameter_opt


Fitted LGBMClassifier
Fold: 0	 AUC: 0.77603247991778

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7743390699409278

Fitted LGBMClassifier
Fold: 2	 AUC: 0.7748151740490261

Fitted LGBMClassifier
Fold: 3	 AUC: 0.7717161233720741

Fitted LGBMClassifier
Fold: 4	 AUC: 0.7771396746210494

Fitted LGBMClassifier
Fold: 5	 AUC: 0.7770258485732717

Fitted LGBMClassifier
Fold: 6	 AUC: 0.7755892276950306

Fitted LGBMClassifier
Fold: 7	 AUC: 0.7743321366352652

Fitted LGBMClassifier
Fold: 8	 AUC: 0.7768273620128942

Fitted LGBMClassifier


[I 2021-03-22 14:10:54,984] Trial 0 finished with value: 0.7748517129890805 and parameters: {'random_state': 54, 'reg_alpha': 3.103717893483926e-07, 'reg_lambda': 0.001309168497610965, 'max_depth': 4, 'n_estimators': 444, 'num_leaves': 204, 'colsample_bytree': 0.43956080673594555, 'subsample': 0.9894353835585619, 'subsample_freq': 7, 'min_child_samples': 46}. Best is trial 0 with value: 0.7748517129890805.


Fold: 9	 AUC: 0.7707000330734854

Average AUC: 0.7707000330734854
Fitted LGBMClassifier
Fold: 0	 AUC: 0.7662981016483555

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7657248456646158

Fitted LGBMClassifier
Fold: 2	 AUC: 0.7642956260188394

Fitted LGBMClassifier
Fold: 3	 AUC: 0.7589821859353484

Fitted LGBMClassifier
Fold: 4	 AUC: 0.7639102084075216

Fitted LGBMClassifier
Fold: 5	 AUC: 0.7616060398256839

Fitted LGBMClassifier
Fold: 6	 AUC: 0.761184083978846

Fitted LGBMClassifier
Fold: 7	 AUC: 0.7618424969341093

Fitted LGBMClassifier
Fold: 8	 AUC: 0.7640527035315531

Fitted LGBMClassifier


[I 2021-03-22 14:34:05,328] Trial 1 finished with value: 0.7627437947974908 and parameters: {'random_state': 43, 'reg_alpha': 2.1716166620751105, 'reg_lambda': 1.697400524470599e-05, 'max_depth': 1, 'n_estimators': 278, 'num_leaves': 113, 'colsample_bytree': 0.9712223303913177, 'subsample': 0.7443715832551345, 'subsample_freq': 3, 'min_child_samples': 100}. Best is trial 0 with value: 0.7748517129890805.


...

Fold: 9	 AUC: 0.7761925066173931

Average AUC: 0.7761925066173931
Fitted LGBMClassifier
Fold: 0	 AUC: 0.7811765704439491

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7797954730752522

Fitted LGBMClassifier
Fold: 2	 AUC: 0.777970187630202

Fitted LGBMClassifier
Fold: 3	 AUC: 0.7756007832044682

Fitted LGBMClassifier
Fold: 4	 AUC: 0.7782644508179372

Fitted LGBMClassifier
Fold: 5	 AUC: 0.7799550817660998

Fitted LGBMClassifier
Fold: 6	 AUC: 0.7767038236307646

Fitted LGBMClassifier
Fold: 7	 AUC: 0.7754587560072386

Fitted LGBMClassifier
Fold: 8	 AUC: 0.7779562924867549

Fitted LGBMClassifier


[I 2021-03-22 20:14:36,377] Trial 33 finished with value: 0.7777384808808194 and parameters: {'random_state': 41, 'reg_alpha': 9.105395525604685e-06, 'reg_lambda': 0.6320956492490926, 'max_depth': 10, 'n_estimators': 461, 'num_leaves': 135, 'colsample_bytree': 0.8660654205739193, 'subsample': 0.8138277897176545, 'subsample_freq': 5, 'min_child_samples': 89}. Best is trial 22 with value: 0.7786212639623306.


Fold: 9	 AUC: 0.7745033897455281

Average AUC: 0.7745033897455281
Fitted LGBMClassifier
Fold: 0	 AUC: 0.7799065200943404

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7784615621300311

Fitted LGBMClassifier
Fold: 2	 AUC: 0.777891085174981

Fitted LGBMClassifier
Fold: 3	 AUC: 0.7752422200264409

Fitted LGBMClassifier
Fold: 4	 AUC: 0.777403864952125

Fitted LGBMClassifier
Fold: 5	 AUC: 0.7800739979444545

Fitted LGBMClassifier
Fold: 6	 AUC: 0.7790579576647676

Fitted LGBMClassifier
Fold: 7	 AUC: 0.7772997797708213

Fitted LGBMClassifier
Fold: 8	 AUC: 0.7771572561146677

Fitted LGBMClassifier


[I 2021-03-22 20:21:11,206] Trial 34 finished with value: 0.7776556077510413 and parameters: {'random_state': 30, 'reg_alpha': 0.00046742618080696435, 'reg_lambda': 8.018427327396678, 'max_depth': 8, 'n_estimators': 379, 'num_leaves': 179, 'colsample_bytree': 0.8021309331006635, 'subsample': 0.8574056191501539, 'subsample_freq': 7, 'min_child_samples': 72}. Best is trial 22 with value: 0.7786212639623306.


Fold: 9	 AUC: 0.7740618336377837

Average AUC: 0.7740618336377837
Fitted LGBMClassifier
Fold: 0	 AUC: 0.7798981830082723

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7787886772031153



Best Params: 
    
    reg_alpha: 0.362136938773081
    reg_lambda: 2.930297242488071
    max_depth: 10
    n_estimators: 306
    num_leaves: 71
    colsample_bytree: 0.7121396258381646
    subsample: 0.793959734582999
    subsample_freq: 2
    min_child_samples: 18

In [10]:
if OPTUNA_OPTIMIZATION:
    final_model = LGBMClassifier(**trial.params)
else:
    final_model = LGBMClassifier(**trial)

In [11]:
test_preds = []

skf = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 29)
aucs = []
for kfold, (train_idx, val_idx) in enumerate(skf.split(train_preprocessed[features].values, 
                                                      train_preprocessed['target'].values)):
        
        final_model.fit(train_preprocessed.loc[train_idx, features], train_preprocessed.loc[train_idx, 'target'])
        print('Fitted {}'.format(type(final_model).__name__))
        val_true = train.loc[val_idx, 'target'].values
        
        preds = final_model.predict(train_preprocessed.loc[val_idx, features])
        
        auc = roc_auc_score(val_true, preds)
        aucs.append(auc)
        print('Fold: {}\t Validation AUC: {}\n'.format(kfold, auc))
        
        test_preds.append(final_model.predict_proba(test_preprocessed[features])[:, 1])
        
print("Best Parameters mean AUC: {}".format(np.mean(aucs)))

Fitted LGBMClassifier
Fold: 0	 Validation AUC: 0.7813630335680872

Fitted LGBMClassifier
Fold: 1	 Validation AUC: 0.7797455076230869

Fitted LGBMClassifier
Fold: 2	 Validation AUC: 0.7801258351038276

Fitted LGBMClassifier
Fold: 3	 Validation AUC: 0.7765705729142831

Fitted LGBMClassifier
Fold: 4	 Validation AUC: 0.7798306360624886

Fitted LGBMClassifier
Fold: 5	 Validation AUC: 0.7797695544955657

Fitted LGBMClassifier
Fold: 6	 Validation AUC: 0.778813192002396

Fitted LGBMClassifier
Fold: 7	 Validation AUC: 0.7770425227454083

Fitted LGBMClassifier
Fold: 8	 Validation AUC: 0.7792939183142692

Fitted LGBMClassifier
Fold: 9	 Validation AUC: 0.7769395743604536

Best Parameters mean AUC: 0.7789494347189867


<a id = "submission"></a>

### Submission

In [12]:
test_predictions = np.mean(test_preds, axis = 0)

In [13]:
len(test_predictions) == len(test)

True

In [14]:
sample_submission['target'] = test_predictions

sample_submission.to_csv("submission.csv", index = False)